In [ ]:
---------------------------------------------------------------------
--1. List Customers With Sales Above the Average Customer Sales

--Functional Specification:

--Sum sales per customer for Sales.Orders → Sales.OrderLines.

--Compare each customer’s total to average total per customer (subquery).

--Return customers whose total > global average; order by total desc.
---------------------------------------------------------------------
SELECT c.CustomerName, sum(ol.UnitPrice * ol.Quantity) as SalesSum
FROM Sales.Customers c
JOIN Sales.Orders o 
ON c.CustomerID = o.CustomerID
JOIN Sales.OrderLines ol 
ON o.OrderID = ol.OrderID
GROUP BY c.CustomerName
HAVING sum(ol.UnitPrice * ol.Quantity) > (
	SELECT AVG(t.SalesSum)
	FROM (
        SELECT o.CustomerID, sum(ol.UnitPrice * ol.Quantity) as SalesSum
		FROM Sales.Orders o
		INNER JOIN Sales.OrderLines ol 
		ON o.OrderID = ol.OrderID
		GROUP BY o.CustomerID
    ) as t
)
ORDER BY SalesSum DESC

---------------------------------------------------------------------
--2. List Customers Who Has At Least 1 Order That Contain at Least 3 Distinct Items 

--Functional Specification:

--For each order, check the ammount of distinct positions by subquery(Sales.OrderLines).

--Return Customers With DISTINCT StockItemID >= 3.
---------------------------------------------------------------------
SELECT c.CustomerName, o.OrderID
FROM Sales.Customers c
JOIN Sales.Orders o 
ON c.CustomerID = o.CustomerID
JOIN (
    SELECT ol.OrderID, COUNT(DISTINCT ol.StockItemID) as NumOfOrders
    FROM Sales.OrderLines ol
    GROUP BY ol.OrderID
    HAVING COUNT(DISTINCT ol.StockItemID) > 3
) ol
ON o.OrderID = ol.OrderID

---------------------------------------------------------------------
--3. List Top-3 Suppliers by Number of Items 

--Functional Specification:

--Count The Number Of Orders For each Supplier(Purchasing.PurchaseOrders).

--Join With Purchasing.Suppliers For Names.
---------------------------------------------------------------------
SELECT s.SupplierName, NumOfOrders
FROM Purchasing.Suppliers s 
JOIN (
	SELECT TOP (3) po.SupplierID, COUNT(po.PurchaseOrderID) as NumOfOrders
	FROM Purchasing.PurchaseOrders po 
	GROUP BY po.SupplierID
	ORDER BY COUNT(po.PurchaseOrderID) DESC
) p
ON s.SupplierID = p.SupplierID

---------------------------------------------------------------------
--4. Show Last Order per Customer (Top 20 customers)

--Functional Specification:

--For each customer, find the MAX(OrderDate) using a correlated subquery for Sales.Orders

--Return the customer ID, order ID and the date of their most recent order

--Sort by date 
---------------------------------------------------------------------
SELECT TOP (20)
    o.CustomerID,
    o.OrderID,
    o.OrderDate
FROM Sales.Orders AS o
WHERE o.OrderDate = (
    SELECT MAX(o2.OrderDate)
    FROM Sales.Orders AS o2
    WHERE o2.CustomerID = o.CustomerID  
)
ORDER BY o.OrderDate DESC;

---------------------------------------------------------------------
--5. Show Customers Who Spent More Than The Average Customer

--Functional Specification:

--Calculate global average spending per customer (declare a variable).

--Sum total sales per customer using Sales.Customers, Sales.Orders, Sales.OrderLines.

--Return customers whose spending > average; order by total descending.
---------------------------------------------------------------------
DECLARE @averagespending AS INT = (
	SELECT AVG(Quantity * UnitPrice) FROM Sales.OrderLines
);
                         
SELECT c.CustomerName, SUM(ol.Quantity * ol.UnitPrice) AS TotalSpent
FROM Sales.Customers AS c
JOIN Sales.Orders AS o ON c.CustomerID = o.CustomerID
JOIN Sales.OrderLines AS ol ON o.OrderID = ol.OrderID
GROUP BY c.CustomerName
HAVING SUM(ol.Quantity * ol.UnitPrice) > @averagespending;

---------------------------------------------------------------------
--6. Count the number of orders yearly

--Functional Specification:

--Extract YEAR(OrderDate) and CustomerID from Sales.Orders.

--Group by year and count distinct customers.
---------------------------------------------------------------------
SELECT OrderYear, COUNT(DISTINCT CustomerID) AS NumOfOrders
FROM (SELECT YEAR(OrderDate) AS OrderYear, CustomerID
      FROM Sales.Orders) AS D
GROUP BY OrderYear;

---------------------------------------------------------------------
--7. Show Customers With Total Sales per Year 

--Functional Specification:

--Create derived table with yearly customer totals from Sales.Orders and Sales.OrderLines.

--Join to Sales.Customers to show their yearly sales.
---------------------------------------------------------------------
SELECT c.CustomerName, dt.OrderYear, dt.TotalSales
FROM Sales.Customers AS c
JOIN (
    SELECT CustomerID, YEAR(OrderDate) AS OrderYear,
           SUM(Quantity * UnitPrice) AS TotalSales
    FROM Sales.Orders AS o
    JOIN Sales.OrderLines AS ol ON o.OrderID = ol.OrderID
    GROUP BY CustomerID, YEAR(OrderDate)
) AS dt ON c.CustomerID = dt.CustomerID;

---------------------------------------------------------------------
--8. List Customers With Invoices Count 

--Functional Specification:

--Create derived table with count of invoices per customer from Sales.Invoices.

--Return all customers with that count from Sales.Customers.
---------------------------------------------------------------------
SELECT c.CustomerName, d.NumberOfInvoices
FROM Sales.Customers AS c
JOIN (
    SELECT CustomerID, COUNT(*) AS NumberOfInvoices
    FROM Sales.Invoices
    GROUP BY CustomerID
) AS d ON c.CustomerID = d.CustomerID;

---------------------------------------------------------------------
--9. Show Top 3 Suppliers by Orders From Purchasing

--Functional Specification:

--Create derived table with count of purchase orders per supplier from Purchasing.PurchaseOrders.

--Select top 3 suppliers by total orders from Purchasing.Suppliers.
---------------------------------------------------------------------
SELECT TOP (3) s.SupplierName, dt.NumerOfOrders
FROM Purchasing.Suppliers AS s
JOIN (
    SELECT SupplierID, COUNT(*) AS NumerOfOrders
    FROM Purchasing.PurchaseOrders
    GROUP BY SupplierID
) AS dt ON s.SupplierID = dt.SupplierID
ORDER BY NumerOfOrders DESC;

---------------------------------------------------------------------
--10. List Stock Value by Product At Warehouse

--Functional Specification:

--Calculate stock value per item = Quantity * UnitCost from Warehouse.StockItemHoldings.

--Return top 10 by total value with Warehouse.StockItems.
---------------------------------------------------------------------
SELECT TOP (10) p.StockItemName, d.StockValue
FROM Warehouse.StockItems AS p
JOIN (
    SELECT StockItemID, SUM(QuantityOnHand * LastCostPrice) AS StockValue
    FROM Warehouse.StockItemHoldings
    GROUP BY StockItemID
) AS d ON p.StockItemID = d.StockItemID
ORDER BY d.StockValue DESC;